<a href="https://colab.research.google.com/github/silverstar0727/study-/blob/master/_posts/NN/CNN_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN

Convolutional Neural Network 원리 및 구현

우선적으로 합성곱 신경망에 대해서.. - 합성곱 신경망은 이미지 분류 작업에서 탁월한 성능을 보인다!!

신경망 = 유용한 데이터 특성을 자동적으로 학습하는 기능을 가지고 있음!! 따라서 특성을 추출할때 용이한데, 다층(심층 합성곱) 신경망은 저수준 특성들을 연결하여 고수준 특성을 만드는 역할을 수행함 
ex) 동그라미 -> 강아지

cnn은 두 가지의 중요한 특성으로 인한 이미지 분류가 뛰어나다
> 1. 희소연결: 특성맵(feature map)에 있는 하나의 원소는 하나의 픽셀 패치(patch)에만 연결된다.
2. 파라미터 공유: 동일한 가중치가 입력 이미지의 모든 패치에 사용된다.



## CNN의 구성
여러개의 합성곱(Conv)층 + 풀링(Pooling)(subsampling)층 + 완전 연결(FC)층으로 구성 되어 있다.

완전 연결층은 다층 퍼셉트론으로 구성되어 있다.

### 이산 합성곱(discrete convolution)
#### 1차원 합성곱


합성공의 수학적 식에서 특이할 점은 두 가지가 존재한다.
* 인덱싱의 범위가 -무한대 ~ +무한대
* x의 음수 인덱싱

따라서, 인덱싱의 범위를 유한한 개수로 패딩(padding)을 해야한다. 패딩의 종류는 

1. 풀(full) 패딩 (p = m - 1)
2. 세임(same) 패딩 (동일하게 케바케)
3. 밸리드(valid) 패딩 (p = 0)

존재하고, 세임 패딩은 데이터의 크기가 변하지 않아서 가장 많이 사용된다. 방식은 테두리에 0을 집어넣는 방식으로 패딩한다.



In [0]:
# 1차원 합성곱 단순 구현
import numpy as np

def conv1(x, w, p = 0, s = 1): # x = 입력 데이터, w = 필터 or 커널, p = 패딩크기, s = 필터의 이동간격
  # 패딩이 존재하지 않을 때
  w_rot = np.array(w[::-1]) # 필터를 역순으로
  x_padded = np.array(x) # 입력 데이터

  if p > 0: # 패딩이 존재할 때
    zero_pad = np.zeros(shape = p) # [0, 0, 0, .. p개]
    x_padded = np.concatenate([zero_pad, x_padded, zero_pad]) # 제로 패딩

  res = [] # 합성곱의 결과 값
  for i in range(0, int(len(x)/s), s): # 필터 이동...
    res.append(np.sum(x_padded[i : i + w_rot.shape[0]] * w_rot)) # 결과를 새 행렬에 하나씩 추가함
  
  return np.array(res)

In [0]:
## test
x = [1,3,2,4,5,6,1,3]
w = [1,0,3,1,2]

print('Conv1 구현:', conv1(x, w, p = 2, s = 1))

Conv1 구현: [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [0]:
# numpy로 합성곱 수행
print('넘파이 결과', np.convolve(x, w, mode = 'same'))

넘파이 결과 [ 5 14 16 26 24 34 19 22]


#### 2차원 합성곱
2차원도 1차원과 동일하게 수행한다.

(필터를 역전시키기 위해서는 w_rot = w[::-1, ::-1]과 같이 사용한다.)

In [0]:
# 2차원 합성곱 단순 구현
import numpy as np
import scipy.signal

def conv2d(X, W, p = (0,0), s = (1,1)): # p = (0,0)
  W_rot = np.array(W)[::-1, ::-1] # W역순으로 만들기
  X_orig = np.array(X) # X 입력 데이터
  n1 = X_orig.shape[0] + 2*p[0] # 패딩하는 1차원 크기
  n2 = X_orig.shape[1] + 2*p[1] # 패딩하는 2차원 크기
  X_padded = np.zeros(shape = (n1, n2)) # 제로 행렬 만들기
  X_padded[p[0] : p[0] +X_orig.shape[0], p[1]: p[1] + X_orig.shape[1]] = X_orig # 제로 행렬 안에 X 데이터 넣기

  res = [] # 결과 행렬
  for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) / s[0]) + 1, s[0]): # 1차원 방향으로 움직이기
    res.append([]) # 1차원 뱡향 반복시마다 차원확장
    for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) / s[1]) + 1, s[1]): # 2차원 방향으로 움직이기
      X_sub = X_padded[i : i + W_rot.shape[0], j : j + W_rot.shape[1]] # 계산하기

      res[-1].append(np.sum(X_sub * W_rot)) # 마지막에 계산값 추가

  return (np.array(res))

In [0]:
## test

X = [[1,3,2,4], [5,6,1,3], [1,2,0,2], [3,4,3,2]]
W = [[1,0,3], [1,2,1], [0,1,1]]

print('Conv2d 구현:', conv2d(X, W, p = (1,1), s = (1,1)))

Conv2d 구현: [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [0]:
# scipy로 구현
scipy.signal.convolve2d(X, W, mode = 'same')

array([[11, 25, 32, 13],
       [19, 25, 24, 13],
       [13, 28, 25, 17],
       [11, 17, 14,  9]])